In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import json

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## NASA Mars News

In [11]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [12]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")
news_list = soup.find("ul", class_= "item_list")

news_title = news_list.find('div',class_="content_title").get_text()
print(news_title)

NASA's Perseverance Rover Will Carry First Spacesuit Materials to Mars


In [13]:
news_p = soup.find('div', class_='article_teaser_body').get_text()

print(news_p)

In a Q&A, spacesuit designer Amy Ross explains how five samples, including a piece of helmet visor, will be tested aboard the rover, which is targeting a July 30 launch. 


## JPL Mars Space Images - Featured Image

In [6]:
url_base = 'https://www.jpl.nasa.gov'
url = f'{url_base}/spaceimages/?search=&category=Mars'
print(url)
browser.visit(url)

https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars


In [7]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
image_div = soup.find(id= 'full_image')

image_url = image_div['data-fancybox-href']

featured_image_url = f'{url_base}{image_url}'
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18297_ip.jpg


## Mars Weather

In [8]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

In [10]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

mars_twitter = soup.find("div",{"data-testid": "tweet"})
mars_weather = mars_twitter.find(class_="css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0").get_text()

print(mars_weather)


InSight sol 594 (2020-07-28) low -91.8ºC (-133.2ºF) high -16.6ºC (2.2ºF)
winds from the WNW at 8.0 m/s (17.8 mph) gusting to 20.7 m/s (46.4 mph)
pressure at 7.90 hPa


## Mars Facts

In [10]:
# Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
html_link = "https://space-facts.com/mars/"
mars_facts=pd.read_html(html_link)

mars_facts_df = mars_facts[0]
mars_facts_df.columns = ['Mars Planet Profile', 'Measurement']
mars_facts_df

,Mars Planet Profile,Measurement
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [11]:
#  Pandas to convert the data to a HTML table string
mars_facts_html = mars_facts_df.to_html(index=False)
mars_facts_html_final = mars_facts_html.replace('\n', '')
mars_facts_html_final

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>Mars Planet Profile</th>      <th>Measurement</th>    </tr>  </thead>  <tbody>    <tr>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

## Mars Hemispheres

In [12]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [13]:
# Hemisphere titles list 

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
image_titles = soup.find_all('h3')

img_title_list = []
for title in image_titles:
    img_title_list.append(title.text)

print(img_title_list)

# Hemisphere pages to obtain the url images  
pages_for_url = soup.find_all('div',class_='description')

print('----------------------------------------------------------------------------------------------------------')
print(pages_for_url)

['Cerberus Hemisphere Enhanced', 'Schiaparelli Hemisphere Enhanced', 'Syrtis Major Hemisphere Enhanced', 'Valles Marineris Hemisphere Enhanced']
----------------------------------------------------------------------------------------------------------
[<div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div>, <div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 35 MB</span><span class="pubDate" style="float:right"></

In [14]:
# Hemisphere url images list

image_urls = []
for image in pages_for_url:
    link = image.a['href']
    url = f'https://astrogeology.usgs.gov{link}'
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    image_url_soup = soup.find('img', class_= 'wide-image')
    url_append = image_url_soup['src']
    image_urls.append(f'https://astrogeology.usgs.gov{url_append}')

print(image_urls)
   

['https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg', 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg', 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg', 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']


In [15]:
# Json with hemisphere titles and images 

hemisphere_image_urls = []

for i in range(0,len(img_title_list)):
    hemisphere_image_urls.append({"title" : img_title_list[i], "img_url" : image_urls[i]})

json.dumps(hemisphere_image_urls, indent = 1)

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [18]:
# Dictionary with all the scraped information to return 
all_scraping = {}

all_scraping.update({"news_title":news_title,"news_text":news_p,"feature_img":featured_image_url,
                    "mars_weather_desc":mars_weather,"facts_table":mars_facts_html_final,"img_title_list":img_title_list,
                     "image_urls":image_urls})


In [19]:
all_scraping

{'news_title': "A New Video Captures the Science of NASA's Perseverance Mars Rover",
 'news_text': 'With a targeted launch date of July 30, the next robotic scientist NASA is sending to the to the Red Planet has big ambitions.',
 'feature_img': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA20263_ip.jpg',
 'mars_weather_desc': 'InSight sol 591 (2020-07-25) low -91.2ºC (-132.2ºF) high -15.5ºC (4.2ºF)\nwinds from the WNW at 7.5 m/s (16.9 mph) gusting to 19.0 m/s (42.5 mph)\npressure at 7.90 hPa',
 'facts_table': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>Mars Planet Profile</th>      <th>Measurement</th>    </tr>  </thead>  <tbody>    <tr>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr> 